In [2]:
import numpy as np
import pandas as pd
df=pd.read_csv('data/train.csv',sep="\t")
df = df.drop('Unnamed: 0', 1)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,140593,7629,yakusho alway wonder longfac sad sack hi chemi...,4
1,93544,4877,flick guy,2
2,31590,1479,countless,2
3,13873,596,earn uplift,3
4,152452,8319,no clear pictur kill bob crane,1


Replace empty phrase rows to NaN then delete.

In [3]:
#print("size of data with empty phrase rows=",len(df.index))

#define a map to change empty phrases to NaN
def change_nan(x):
    if pd.isnull(x):
        return np.nan 
    if len(x) == 0:
        return np.nan
    return x

df['Phrase'] = df.Phrase.map(change_nan)

#remove NaN rows of df
df=df.dropna()

#reset indexing
df=df.reset_index(drop=True)

#print("size of data after empty phrase rows deleted=",len(df.index))

Afer clean, the same same phrases with differen sentiment scores created as follows.

In [4]:
for x in df.Phrase:
    same=df.Phrase[df.Phrase==x]
    if len(same.index)>1:
        break
df.loc[same.index[0]]

PhraseId                                                 140593
SentenceId                                                 7629
Phrase        yakusho alway wonder longfac sad sack hi chemi...
Sentiment                                                     4
Name: 0, dtype: object

In [5]:
df.loc[same.index[1]]

PhraseId                                                 140594
SentenceId                                                 7629
Phrase        yakusho alway wonder longfac sad sack hi chemi...
Sentiment                                                     3
Name: 26006, dtype: object

Take a phrase as referance to the "phrase" list. Find the same ones in data and record the indexes to the "same" list. In such same phrase groups assign the first one as referance by changing its sentiment score to group average and change other phrases to NaN to remove later from the list. Before doing that sentiment scores should be converted into float data type from intiger(as a default).

In [6]:
#print("data type of sentiment scores= ",df.Sentiment.dtype)

#convert sentiment scores to float to write averaged ones
df.Sentiment=df.Sentiment.astype(float)
df.dtypes
#print("data type of sentiment scores after conversion= ",df.Sentiment.dtype)

#write size of data before merge
#print("data size before merge= ",len(df.index))

#"phrase" holds the current phrase to compare through whole set and find the indexes of same other phrase rows 
#which will written in "same"
#then they will changed to NaN for further clean process
#summation holds the summation of same phrase sentiment scores that will avaraged and written to the first phrase among all same phrases
same=[]
summation=0
phrase=[]

for index,row in df.iterrows():
    if row.Phrase!=np.nan :
        same=df.Phrase[df.Phrase==row.Phrase]
        if len(same.index)>1:
            summation=0
            phrase=df.loc[same.index[0],'Phrase']
            for x in range(1,len(same.index)):
                df.loc[same.index[x],'Phrase']=np.nan
                summation+=df.loc[same.index[x],'Sentiment']
            summation+=df.loc[same.index[0],'Sentiment']
            df.loc[same.index[0],'Sentiment']=summation/len(same.index)

#clean all NaN rows
df=df.dropna()

#write size of data after merge
#print("data size after merge= ",len(df.index))

Remove NaN assigneds and reindex data.

In [7]:
df=df.dropna()
df=df.reset_index(drop=True)
df.head(15)

,PhraseId,SentenceId,Phrase,Sentiment
0,140593,7629,yakusho alway wonder longfac sad sack hi chemi...,3.500000
1,93544,4877,flick guy,2.000000
2,31590,1479,countless,2.000000
3,13873,596,earn uplift,3.000000
4,152452,8319,no clear pictur kill bob crane,1.750000
5,96410,5036,attempt make film relev today without fulli un...,2.000000
6,84521,4373,fairi tale,2.250000
7,122338,6557,not tick hum,3.000000
8,99668,5228,alli,2.500000
9,153089,8360,theori,2.333333


This cleaned data is sort of vocabulary list. Save file as train-clean.csv under data folder.

In [8]:
df.to_csv("data/train-clean.csv", sep='\t')
df.describe()

,PhraseId,SentenceId,Sentiment
count,75819.000000,75819.000000,75819.000000
mean,79360.938116,4152.322320,2.063092
std,44736.596744,2488.383312,0.880306
min,1.000000,1.000000,0.000000
25%,40970.000000,1964.000000,1.666667
50%,79814.000000,4112.000000,2.000000
75%,118160.500000,6312.000000,2.666667
max,156060.000000,8544.000000,4.000000


In [9]:
print("done succesfully")

done succesfully
